# Spark Streaming

In [2]:
!pip install findspark --quiet

You should consider upgrading via the 'c:\users\amit.kumar02\miniconda3\envs\poc\python.exe -m pip install --upgrade pip' command.


In [14]:
!pip install tweepy==3.10.0 --quiet

You should consider upgrading via the 'c:\users\amit.kumar02\miniconda3\envs\poc\python.exe -m pip install --upgrade pip' command.


## Run Tweets.py

In [1]:
import findspark

In [2]:
%ls 

 Volume in drive C is System
 Volume Serial Number is EE4C-9909

 Directory of C:\Amit\GIT\Streaming\PySpark

03/17/2022  03:44 PM    <DIR>          .
03/17/2022  03:44 PM    <DIR>          ..
03/17/2022  03:37 PM    <DIR>          .ipynb_checkpoints
03/17/2022  03:28 PM               553 envsetup.bat
03/17/2022  03:28 PM               553 envsetup.sh
03/17/2022  03:37 PM            39,711 First Twitter App-orig.ipynb
03/17/2022  03:28 PM               611 notes.txt
03/17/2022  03:37 PM    <DIR>          spark-warehouse
03/17/2022  03:28 PM               207 test.py
03/17/2022  03:28 PM             1,713 TweetRead.py
03/17/2022  03:37 PM             4,081 TweetRead-orig.ipynb
03/17/2022  03:28 PM             1,712 TweetRead-orig.py
03/17/2022  03:28 PM             1,847 Twitter_App.py
               9 File(s)         50,988 bytes
               4 Dir(s)  183,651,221,504 bytes free


In [3]:

# findspark.init('/home/ubuntu-poc/workspace/projects/Streaming/PySpark/spark-2.1.0-bin-hadoop2.7')
findspark.init('C:\Amit\GIT\spark-3.2.1-bin-hadoop3.2')

In [4]:
import warnings
warnings.filterwarnings('ignore')

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [5]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

In [6]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [7]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5556)

In [8]:
lines = socket_stream.window( 20 )

In [9]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [10]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count")) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ))  # Registers to a table.

In [11]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [12]:
ssc.start()

In [ ]:
import time 
count = 0
while count < 10:
    
    time.sleep( 3 )
#     Select tag, count from tweets
    top_10_tweets = sqlContext.sql( 'select * from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

In [16]:
ssc.stop()

22/03/12 15:46:26 WARN ReceiverSupervisorImpl: Receiver has been stopped
Exception in thread "receiver-supervisor-future-2" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurren